In [3]:
from database import sql

In [4]:
def execute(language, name, parameter = None):
    db = sql('functional')
    
    base = db.query(f'''
        SELECT
            *
        FROM
            command
        WHERE
            language = '{language}'
        AND
            name = '{name}'
    ''')[0]

    def next(id_child):
        child = db.query(f'''
            SELECT
                *
            FROM
                sub_command
            WHERE
                parent = {id_child}
        ''')
        ret = ''
        for i in child:

            child = i['child']
            id = i['id']

            data = db.query(f'''
                SELECT
                    *
                FROM
                    command
                WHERE
                    id = {child}
            ''')[0]

            cur = data['value'] + next(child)

            
            parameter = db.query(f'''
                SELECT
                    *
                FROM
                    sub_command_parameter
                WHERE
                    sub_command = {id}
            ''')

            for i in parameter:
                cur = cur.replace('{{'+i['parameter']+'}}', next(i['command']) if i['command'] else i['value'])

            ret += cur + '\n'
        return ret
    
    ret = base['value'] + next(base['id'])
    if(parameter):
        for i in parameter:
            ret = ret.replace('{{'+i+'}}',parameter[i])
    db.close()
    return ret

In [5]:
print(execute('sql','select',{'field':'0'}))

SELECT
    0


In [6]:
print(execute('sql','test'))

ProgrammingError: 1054 (42S22): Unknown column 'None' in 'where clause'